<!--
%\VignetteEngine{knitr::rmarkdown}
%\VignetteIndexEntry{lmer Performance Tips}
-->

In [1]:
library("knitr")
knitr::opts_chunk$set(
  eval = FALSE
)

In [2]:
library("lme4")

Loading required package: Matrix



# lme4 Performance Tips

- use control = `[g]lmerControl(calc.derivs = FALSE)` to turn off the time
  consuming derivative calculation that is performed after the optmization
  is finished, e.g.

In [3]:
lmer(y ~ service * dept + (1|s) + (1|d), InstEval,
     control = lmerControl(calc.derivs = FALSE))

Linear mixed model fit by REML ['lmerMod']
Formula: y ~ service * dept + (1 | s) + (1 | d)
   Data: InstEval
REML criterion at convergence: 237688.7
Random effects:
 Groups   Name        Std.Dev.
 s        (Intercept) 0.3250  
 d        (Intercept) 0.5122  
 Residual             1.1768  
Number of obs: 73421, groups:  s, 2972; d, 1128
Fixed Effects:
    (Intercept)         service1            dept5           dept10  
       3.229529         0.252047         0.129657        -0.176588  
         dept12            dept6            dept7            dept4  
       0.051739         0.034960         0.145994         0.151835  
          dept8            dept9           dept14            dept1  
       0.104311         0.044171         0.051898         0.046482  
          dept3           dept11            dept2   service1:dept5  
       0.056541         0.059799         0.005642        -0.180963  
service1:dept10  service1:dept12   service1:dept6   service1:dept7  
       0.018274        -0.2

Note that this will disable some of the convergence tests, as well as
(for `glmer` only) making `lme4` use a less
accurate approximation to compute the standard errors of the fixed effects.

- models that only contain random effects of the form `(1|f)` use better 
  starting values for the optimization which in tests have cut run time in 
  certain examples by up to 50% relative to the previous default starting 
  values.  The `InstEval` fit shown above is one such example.

- `lmer` uses the `bobyqa` optimizer from the `minqa` package by default;
  `glmer` uses a combination of Nelder-Mead and `bobyqa`.  If you
  are specifying the `optimx` package optimizer, note that by 
  default `optimx` performs 
  certain time-consuming processing at the beginning and end which can be 
  turned off as follows (here we have specified the `"nlminb"` method but this 
  applies to any `optimx` method):

- the `nloptr` package supports a variety of algorithms and importantly 
  supports additional stopping criteria which can stop the optimization
  earlier if it believes it has reached the optimum.
  For many problems using these stopping 
  criteria will result in the 
  same solution or nearly the same solution as the default optimizer but in
  less time (up to 50 percent savings have been observed); however, in
  some cases it may stop prematurely giving suboptimal results.
  (In the example below omit `print_level` if output tracing is not desired and 
  increase `maxeval` if the optimization requires more than 1000 iterations
  and you wish to allow it to proceed.)

In [5]:
nlopt <- function(par, fn, lower, upper, control) {
    .nloptr <<- res <- nloptr(par, fn, lb = lower, ub = upper, 
        opts = list(algorithm = "NLOPT_LN_BOBYQA", print_level = 1,
        maxeval = 1000, xtol_abs = 1e-6, ftol_abs = 1e-6))
    list(par = res$solution,
         fval = res$objective,
         conv = if (res$status > 0) 0 else res$status,
         message = res$message
    )
}
lmer(y ~ service * dept + (1|s) + (1|d), InstEval,
    control = lmerControl(optimizer = "nloptwrap", calc.derivs = FALSE))

Linear mixed model fit by REML ['lmerMod']
Formula: y ~ service * dept + (1 | s) + (1 | d)
   Data: InstEval
REML criterion at convergence: 237688.7
Random effects:
 Groups   Name        Std.Dev.
 s        (Intercept) 0.3250  
 d        (Intercept) 0.5122  
 Residual             1.1768  
Number of obs: 73421, groups:  s, 2972; d, 1128
Fixed Effects:
    (Intercept)         service1            dept5           dept10  
       3.229529         0.252047         0.129657        -0.176588  
         dept12            dept6            dept7            dept4  
       0.051739         0.034960         0.145994         0.151835  
          dept8            dept9           dept14            dept1  
       0.104311         0.044171         0.051898         0.046482  
          dept3           dept11            dept2   service1:dept5  
       0.056541         0.059799         0.005642        -0.180963  
service1:dept10  service1:dept12   service1:dept6   service1:dept7  
       0.018274        -0.2